In [1]:
import matplotlib.pyplot as plt
import sys
import os


In [2]:
# Add the parent directory to the path to import src as a package
sys.path.insert(0, os.path.abspath('..'))
from src import dataloader

In [3]:
plot_flag = False

# Example of loading only the EEG data

In [4]:
# creates a DataLoader class object that creates a structure described in the docs folder (data_structure_spec.md) from raw data
dyad_id = "W_027" #"W003"
lowcut=3.0
highcut=40.0
multimodal_data = dataloader.create_multimodal_data(data_base_path = "../DATA", dyad_id = dyad_id, load_meta=True,load_eeg=False, load_et=False, plot_flag=plot_flag, lowcut=lowcut, highcut=highcut)

No event data: no EEG_events and ET_event columns found in the data.
No events column found in the data.


In [5]:
print(multimodal_data.child_info)
print(multimodal_data.tasks)

ChildInfo(birth_date='03.01.2022', age_years='3l 6m', age_months=np.float64(42.0), age_days=None, rec_date='26-07-2025', group='TD', sex='G')
Tasks(dual_hrv=DualHRV(secore=<WhoEnum.Both: 'Both'>, movies=<WhoEnum.Both: 'Both'>, conversation=None), dual_eeg=DualEEG(movies=<WhoEnum.Both: 'Both'>, conversation=<WhoEnum.Both: 'Both'>), dual_et=DualET(movies=<WhoEnum.CG_Only: 'CG_Only'>, conversation=<WhoEnum.CG_Only: 'CG_Only'>), dual_fnirs=DualFnirs(movies=<WhoEnum.Neither: 'Neither'>, conversation=<WhoEnum.Neither: 'Neither'>))


In [6]:
multimodal_data.data.columns

RangeIndex(start=0, stop=0, step=1)

In [7]:
%matplotlib inline
plt.plot(multimodal_data.data['time'],multimodal_data.data['diode'])
plt.show()

KeyError: 'time'

## Example of direct access to columns in a multimodal_data object

In [ ]:
plt.plot(multimodal_data.data['time'],multimodal_data.data['EEG_cg_Fp1'])
plt.xlim(300, 310)
plt.ylim(-5000, 5000)
plt.show()

In [ ]:
plt.plot(multimodal_data.data['time'],multimodal_data.data['ECG_cg'])
plt.xlim(300, 310)
plt.ylim(-1000, 1000)
plt.show()

# Example of loading just the ET data

In [ ]:
multimodal_data = dataloader.create_multimodal_data(data_base_path = "../data", dyad_id = dyad_id, load_eeg=False, load_et=True, plot_flag=plot_flag, lowcut=lowcut, highcut=highcut)

In [ ]:
multimodal_data.data.columns

In [ ]:
# Create a mapping from unique ET_event values to integers
event_mapping = {event: idx for idx, event in enumerate(multimodal_data.data['ET_event'].unique())}
print("Event mapping:", event_mapping)

# Map ET_event to integer values
multimodal_data.data['event_num'] = multimodal_data.data['ET_event'].map(event_mapping)

plt.plot(multimodal_data.data['time'],multimodal_data.data['event_num'])
plt.show()

# Example of loading both EEG and ET data

In [ ]:
multimodal_data = dataloader.create_multimodal_data(data_base_path = "../data", dyad_id = dyad_id, load_eeg=True, load_et=True, plot_flag=plot_flag, lowcut=lowcut, highcut=highcut)

In [ ]:
multimodal_data.data.columns

## Test of events synchronization between the EEG and ET

In [ ]:

# Create a mapping from unique ET_event values to integers
event_mapping = {event: idx for idx, event in enumerate(multimodal_data.data['ET_event'].unique())}
print("Event mapping:", event_mapping)

# Map ET_event to integer values
multimodal_data.data['event_num_ET'] = multimodal_data.data['ET_event'].map(event_mapping)

# create a mapping from unique events values to integers
event_mapping_events = {event: idx for idx, event in enumerate(multimodal_data.data['events'].unique())}
print("Event mapping for events:", event_mapping_events)

multimodal_data.data['event_num_events'] = multimodal_data.data['events'].map(event_mapping_events)


plt.plot(multimodal_data.data['time_idx']/multimodal_data.fs, multimodal_data.data['event_num_ET'], label='ET events')
plt.plot(multimodal_data.data['time_idx']/multimodal_data.fs    ,multimodal_data.data['event_num_events']+0.5, label='EEG Events')
plt.legend()
plt.xlim(-10,200)
plt.show()

## Example of direct access to the various modalities in the multimodal_data object

In [ ]:
%matplotlib inline
data_selected = multimodal_data.data[(multimodal_data.data.events == 'Brave')]
fig, ax = plt.subplots(figsize=(12, 6),sharex=True, sharey=False,nrows=3)
#ax[0].plot(data_selected['time'], data_selected['ET_cg_y'], linestyle='-', label='ET_cg_y')
ax[0].plot(data_selected['time'], data_selected['ET_ch_y'], linestyle='-', label='ET_ch_y')
ax[0].legend()

#ax[1].plot(data_selected['time'], data_selected['ET_cg_blinks'], linestyle='-', label='ET_cg_blinks')
ax[1].plot(data_selected['time'], data_selected['ET_ch_blinks'], linestyle='-', label='ET_ch_blinks')
ax[1].legend()

#ax[2].plot(data_selected['time'], data_selected['EEG_cg_Fp1'], linestyle='-', label='EEG_cg_Fp1')
ax[2].plot(data_selected['time'], data_selected['EEG_ch_Fp1'], linestyle='-', label='EEG_ch_Fp1')
ax[2].legend()

plt.xlim(0,60)
plt.xlabel('Time')
plt.show()

## Example of access to the same modalities as above using get_signal method

In [ ]:
time, _, ET_ch_y = multimodal_data.get_signals(mode='ET', member='ch', selected_channels=['y'], selected_events=['Brave'])
_, _, ET_ch_blinks = multimodal_data.get_signals(mode='ET', member='ch', selected_channels=['blinks'], selected_events=['Brave'])
_, _, EEG_ch_Fp1 = multimodal_data.get_signals(mode='EEG', member='ch', selected_channels=['Fp1'], selected_events=['Brave'])

fig, ax = plt.subplots(figsize=(12, 6),sharex=True, sharey=False,nrows=3)
ax[0].plot(time, ET_ch_y, linestyle='-', label='ET_ch_y')
ax[0].legend()  
ax[1].plot(time, ET_ch_blinks, linestyle='-', label='ET_ch_blinks')
ax[1].legend()
ax[2].plot(time, EEG_ch_Fp1, linestyle='-', label='EEG_ch_Fp1')
ax[2].legend()
plt.xlabel('Time')
plt.show()
